In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.spatial import distance

pd.set_option('display.max_rows', 500)

### Create inputs

### Populate dataframe

In [2]:
def fill_dataframe(objects, objects_at_once=2):
    df = pd.DataFrame(columns=['from_node', 'to_node', 'obj_at_once', 'dist', 'weight_new'])
    # Generate list of combinations w/o duplicates
    nodes = list(itertools.chain(*[itertools.combinations(objects, i+1) for i in range(len(objects))]))
    nodes = [''.join(node) for node in nodes]
    
    node_list = []
    for x in range(0, len(nodes)):
        if len(nodes[x]) <= 2:
            df.loc[x, 'from_node'] = 'z'
            df.loc[x, 'to_node'] = nodes[x]
            df.loc[x, 'obj_at_once'] = len(nodes[x])
    
        for y in range(x, len(nodes)):
            if 1 <= (len(nodes[y]) - len(nodes[x])) <= 2:
                if all(x in nodes[y] for x in nodes[x]):
                    node_list.append((nodes[y], nodes[x]))
    
    for tup in node_list:
        df = df.append({'from_node': tup[1], 'to_node': tup[0], 'obj_at_once': len(tup[0]) - len(tup[1])}, ignore_index=True)
    
    # No constraints for how many objects at once
    if objects_at_once == 2:
        df = df
    elif objects_at_once == 1:
        # Add constraint: Only 1 object at once
        df = df.loc[df['obj_at_once'] == 1]

    # Reset index
    df = df.reset_index(drop=True)
    
    return df

### Calculate distances

In [3]:
def calculate_distances(data):
    for row in range(0, len(data)):
        # Distance = start -> obj 1 -> table
        if data['from_node'][row] == 'z' and len(data['to_node'][row]) == 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row]], 
                        coordinates['table'])
                    )
    
        # Distance = start -> obj 1 -> obj 2 -> table
        elif data['from_node'][row] == 'z' and len(data['to_node'][row]) != 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row][0]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][0]], 
                        coordinates[data['to_node'][row][1]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][1]], 
                        coordinates['table'])
                    )
    # Distance = table -> obj 1 -> (obj 2) -> table
        else:
            # Get difference between sequences (from_node, to_node)
            diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
            if len(diff) == 1:
                data.loc[row, 'dist'] = distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) * 2
        
            elif len(diff) == 2:
                data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) +
                    distance.euclidean(
                        coordinates[diff[0]], 
                        coordinates[diff[1]]) +
                    distance.euclidean(
                        coordinates[diff[1]], 
                        coordinates['table'])
                    )
    return data

In [4]:
def calculate_edge_weights_params(data, objects, c, k):
    # Reset weights according to weight parameters
    for row in range(0, len(data)):
        diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
        for obj in objects:
            if len(diff) == 1 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                
            elif len(diff) == 2 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                data.loc[row, 'weight_new'] = (data['weight_new'][row] ** k[diff[1]]) * c[diff[1]]
                
    return data

In [9]:
objects = ['r', 'h', 'c', 's', 'e']

coordinates = {'r': (0,5,2),
              'h': (4,4,1),
              'c': (2,0,0),
               's': (4,3,2),
               'e': (4,1,1),
              'start': (4,5,2),
              'table': (2,2,1)}

c1 = {'r': 1.2,
    'h': 1.0,
    'c': 1.2,
    's': 1.2,
    'e': 1.2}

k1 = {'r': 1.0,
    'h': 0.8,
    'c': 0.85,
    's': 1.0,
    'e': 1.0}

c0 = {'r': 1.0,
    'h': 1.0,
    'c': 1.0,
    's': 1.0,
    'e': 1.0}

k0 = {'r': 1.0,
    'h': 1.0,
    'c': 1.0,
    's': 1.0,
    'e': 1.0}

In [26]:
data2 = fill_dataframe(objects, objects_at_once=1)
data2 = calculate_distances(data2)
data2 = calculate_edge_weights_params(data2, objects, c0, k1)
#data2

In [27]:
def get_first(data, objects, c, k):
    possible_items = {}
    
    for row in range(0, len(data)):
        for obj in objects:
            if data.loc[row, 'to_node'] == obj:
                possible_items[obj] = data.loc[row, 'weight_new']
                
    minval = min(possible_items.values())
    list_minval = [k for k, v in possible_items.items() if v==minval]
    
    return possible_items, list_minval
                


In [28]:
get_first(data2, objects, c0, k1)

({'r': 7.741657386773941,
  'h': 3.1776715231464374,
  'c': 5.844288262019726,
  's': 4.449489742783178,
  'e': 6.35917360311745},
 ['h'])

In [10]:
# starting points after action x
start0 = (4,5,2)
start1 = (2,2,1)
start2 = (2,2,1)
start3 = (0,5,2)
start4 = (2,2,1)

In [11]:
def get_next(possible_items, coordinates, c, k, start_coord):
    dist_items = {}
    
    for obj in possible_items:
        dist_items[obj] = ((distance.euclidean(
                        start_coord, 
                        coordinates[obj]) +
                    distance.euclidean(
                        coordinates[obj], 
                        coordinates['table'])
                    ) ** k[obj]) * c[obj]
        
    minval = min(dist_items.values())
    list_minval = [k for k, v in dist_items.items() if v==minval]
        
    return dist_items, list_minval
    

In [29]:
items = ['r', 'c', 's', 'e']
start_coord = start1

get_next(items, coordinates, c0, k1, start_coord)

({'r': 7.483314773547883,
  'c': 3.572216302744254,
  's': 4.898979485566356,
  'e': 4.47213595499958},
 ['c'])

In [30]:
items = ['r', 's', 'e']
start_coord = start2

get_next(items, coordinates, c0, k1, start_coord)

({'r': 7.483314773547883, 's': 4.898979485566356, 'e': 4.47213595499958},
 ['e'])

In [31]:
items = ['r', 's']
start_coord = start3

get_next(items, coordinates, c0, k1, start_coord)

({'r': 3.7416573867739413, 's': 6.9216256977827575}, ['r'])